In [ ]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,LeakyReLU,Input
import tensorflow as tf
from PIL import Image
import numpy as np
import glob
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
import gc
from tensorflow.keras.optimizers import Adam
import pandas as pd #
import json
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
KAGGLE=True
PATH="/kaggle/input/cassava-leaf-disease-classification/" if KAGGLE else "./"
TRAIN_PATH=PATH+"train_images/"
TEST_PATH=PATH+"test_images/"



n_classes=5
img_size=(512,512)
batch_size=32
color_channels=3

top_epochs=3
fine_tuning_epochs=5

In [ ]:
train_df=pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/train.csv')
train_df["label"] = train_df["label"].astype("string")

train_df

In [ ]:
y=pd.get_dummies(train_df.label)
y

def load_image(img_path, resize=img_size):

    pil_img = Image.open(img_path).convert("RGB")
    img = np.asarray(pil_img) / 255
    img_tensor = tf.convert_to_tensor(img)
    img_final = tf.image.resize(img_tensor, resize)
    return img_final.numpy()

def load_imgs(IsTrain=True):
    global PATH
    folder= "train_images/" if IsTrain else "test_images/"
    df= "train.csv" if IsTrain else "sample_submission.csv"
    imgs_path=PATH+folder
    df_path=PATH+df
    data=[]
    for img_id in pd.read_csv(df_path)['image_id']:
        data.append(load_image(imgs_path+img_id))
    return data

def datagen(IsTrain=True):
    global PATH
    global batch_size
    global y
    folder= "train_images/" if IsTrain else "test_images/"
    df= "train.csv" if IsTrain else "sample_submission.csv"
    imgs_path=PATH+folder
    df_path=PATH+df
    data=[]
    train_df=pd.read_csv(df_path)
    y_idx=0
    batchlen=0
    for img_id in train_df['image_id']:
        data.append(load_image(imgs_path+img_id))
        batchlen+=1
        if batchlen==batch_size:
            yield (np.asarray(data),(y.iloc[y_idx:(y_idx+batch_size),:]))
            del data
            y_idx=batch_size+y_idx
            data=[]
            batchlen=0

def train(model,epochs):
    data=datagen()
    for x in range(int(train_df.shape[0]/batch_size)):
        data_batch=data.__next__()
        X,y=data_batch[0],data_batch[1]
        model.fit(X,y,batch_size=batch_size,epochs=epochs)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=90,
    width_shift_range = 0.2,
    height_shift_range = 0.2, 
    horizontal_flip=True,
    fill_mode="nearest",
    vertical_flip=True,
    brightness_range=[0.5,1.5],
    validation_split=0.2

)
    
test_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.2)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    TRAIN_PATH,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    x_col = "image_id",
    y_col = "label",
    shuffle = True,
    seed=42,
    subset = "training"
)

validation_generator = test_datagen.flow_from_dataframe(
    train_df,
    TRAIN_PATH,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    x_col = "image_id",
    y_col = "label",
    shuffle = True,
    subset = "validation",
    seed=42

)

In [ ]:
optimizer = Adam(learning_rate=0.001)
checkpoint = ModelCheckpoint("./best_model", monitor='val_accuracy', verbose=1, save_best_only=True,mode='max')
reduce_lr = ReduceLROnPlateau(
                    monitor='val_loss', 
                    factor=0.5,
                    patience= 2, 
                    verbose = 1,
                    cooldown = 1,
                    min_lr=0.0001)


In [ ]:
new_input = Input(shape=(img_size[0],img_size[1], color_channels))

In [ ]:
base=Xception(weights='imagenet',include_top=False,
                 input_tensor=new_input)
x=base.output
pooling=GlobalAveragePooling2D()(x)
out=Dense(1024)(pooling)
out=LeakyReLU(alpha=0.2)(out)
out=Dense(n_classes,activation="softmax")(out)
model=Model(inputs=base.input,outputs=out)
model.summary()

In [ ]:
for layer in base.layers:
    layer.trainable=False

In [ ]:
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
epochs=10

In [ ]:
history = model.fit(
    train_generator,
    epochs = epochs,
    validation_data = validation_generator,
    verbose = 1,
    callbacks = [
        reduce_lr,
        checkpoint
    ]
)